In [1]:
from thesis_initialise import *

In [2]:
from collections.abc import Mapping as _Mapping
import functools as _functools
import inspect as _inspect
import weakref as _weakref

from everest.h5anchor import disk as _disk
from everest import reseed as _reseed
from everest.funcy import (
    generic as _generic,
    special as _special,
    )
from everest.cascade import (
    Cascade as _Cascade,
    Inputs as _Inputs,
    )
from everest.funcy.base import Base as _Base
from everest import wordhash as _wordhash

In [3]:
myplace = []

import inspect
class A(type):
    def __init__(self, *args, **kwargs):
        print(_inspect.getsource(self.__init__))
        super().__init__(*args, **kwargs)
class B(metaclass = A):
    def __init__(self, a = 1, b = 2):
        ...

    def __init__(self, a = 1, b = 2):
        ...



In [12]:
################################################################################

class Schema(type, _generic.FuncyIncisable):
    def __new__(meta, name, bases, dic, *args, **kwargs):
        cls = super().__new__(meta, name, bases, dic)
        return cls
    def __init__(self, *args, defaults = None, **kwargs):
        if not defaults is None:
            self._defaults = defaults
        super().__init__(*args, **kwargs)
    @property
    def defaults(self):
        try:
            return self._defaults
        except AttributeError:
            defaults = self._defaults = _Inputs(
                self.__init__,
                name = self.__name__
                )
            return defaults
    @property
    def schemaID(self):
        return f'EverestSchema_{self.__name__}'
    @property
    def shape(self) -> tuple:
        return (_special.infint,)
    @property
    def daughter(self) -> type:
        return Case
    def _getitem_strict(self, case):
        return self._get_case(**case)
    def _get_case(self, *args, **kwargs):
        case = _Cascade(*args, **kwargs)
        return self.daughter(
            f"{self.__name__}:{case.hashID}",
            (self, *self.__bases__),
            dict(self.__dict__),
            case = case,
            defaults = self.defaults,
            )
    def _getitem_sub(self, arg):
        raise ValueError("Can't go that deep!")
    def __call__(self, *args, **kwargs):
        return self._get_case(*args, **kwargs)(**kwargs)

class UserSchema(Schema):
    _premade = _weakref.WeakValueDictionary()
    def __new__(meta, *args, **kwargs):
        cls = super().__new__(meta, *args, **kwargs)
        cls._script = script = \
            str(_reseed.digits(12)) # <- TEMPORARY
#             _disk.ToOpen(inspect.getfile(schema))()
        cls._schemaID = schemaID = \
            _wordhash.get_random_proper(2, seed = script)
        try:
            cls = meta._premade[schemaID]
            assert cls.script == script, (script[:32], schema.script[:32])
            raise KeyError
        except KeyError:
            meta._premade[schemaID] = cls
        return cls
    @property
    def script(self): return self._script
    @property
    def schemaID(self): return self._schemaID
    @property
    def daughter(self) -> type:
        return UserCase

class Case(Schema):
    def __new__(meta, *args, **kwargs):
        return Schema.__new__(meta, *args, **kwargs)
    def __init__(self, *args, case: _Cascade, **kwargs):
        self._case = case
        self._inputs = self.defaults.copy(**case)
        super().__init__(*args, **kwargs)
    @property
    def shape(self):
        _, *dims = super().shape
        return tuple(dims)
    @property
    def case(self): return self._case
    @property
    def caseID(self): return self.case.hashID
    @property
    def hashID(self): return self.schemaID + ':' + self.caseID
    @property
    def inputs(self):return self._inputs
    def __call__(self, **kwargs):
        instance = object.__new__(self)
        inputs = self.inputs
        setArgs, setKwargs = inputs.args, inputs.kwargs
        instance.__init__(*setArgs, **{**kwargs, **setKwargs})
        instance.case = self.case
        instanceID = str(_reseed.digits(12))
        instance.instanceID = instanceID
        instance.hashID = self.hashID + ';' + instanceID
        instance.inputs = inputs
        return instance

class UserCase(Case, UserSchema):
    ...

class Basic(metaclass = Schema):
    ...

class MyClass(Basic, metaclass = UserSchema):
    def __init__(self,
            a = 1,
            b = 2,
            c = 3,
            **kwargs,
            ):
        self.foo = a * b * c
        super().__init__(**kwargs)

################################################################################

In [13]:
myobj = MyClass(a = 10)
print(repr(myobj))
print(myobj.hashID)
print(myobj.inputs)
print(myobj.foo)

<__main__.MyClass:insignificant-cattery object at 0x7f1582c00a90>
Glipsy-Mohamedamin:insignificant-cattery;9487504950150
Inputs{MyClass}(
    a = 10
    b = 2
    c = 3
    )
60


In [ ]:
dict(UserSchema._premade)

In [8]:
MyClass

__main__.MyClass

In [14]:
class MyClass2(Basic, metaclass = UserSchema):
    def __init__(self,
            a = 10,
            b = 20,
            c = 30,
            **kwargs,
            ):
        self.boo = a * b * c
        super().__init__(**kwargs)

In [15]:
dict(UserSchema._premade)

{'Glipsy-Mohamedamin': __main__.MyClass, 'Liubomir-Basey': __main__.MyClass2}